### ============================================================================
### Name        : .cpp
### Author      : Muhammad Usman Naveed
### Version     : Final - Release
### Copyright   : (c) Reserved
### Description : Voice Detection Model
### ============================================================================

In [1]:
# !pip install pydub
# !pip install librosa
# !pip install soundfile
# !pip install ffmpeg

### Importing necessary libraraies

In [2]:
import os
from zipfile import ZipFile
import os.path
import shutil 
import pydub
from os import path
from pydub import AudioSegment
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import random
import csv
import pickle

### GETTING THE PATH OF THE CURRENT WORKING DIRECTORY

In [3]:
# GETTING THE PATH OF THE CURRENT WORKING DIRECTORY
path = os.getcwd()
print("Path: ", path)

Path:  C:\Users\Usman Naveed\Desktop\PAI Project


### EXTRACTING THE MAIN ZIP FOLDER

In [4]:
# EXTRACTING THE MAIN ZIP FOLDER
os.chdir(path)
print("Path: ", os.getcwd())
file_name = "PAI_VolunteerTask.zip"

with ZipFile(file_name, 'r') as zip:
#     zip.printdir()
    print('Extracting the main Zip Folder......')
    name_list = []
    zip.extractall()
    name_list = zip.namelist()
    print('Done!')
    
# FIND THE EXTRACTED FOLDER AND MAKE IT THE ROOT PATH 
rootdir = ""
mainFolder = "PAI_VolunteerTask"
for root, subdirs, files in os.walk(os.getcwd()):
    for d in subdirs:
        if d == mainFolder:
            print("Main Folder Found")
            os.chdir(path + "\\" + mainFolder)
            rootdir = os.getcwd()
            break
            
print("ROOT DIRECTORY: ", rootdir)

Path:  C:\Users\Usman Naveed\Desktop\PAI Project
Extracting the main Zip Folder......
Done!
Main Folder Found
ROOT DIRECTORY:  C:\Users\Usman Naveed\Desktop\PAI Project\PAI_VolunteerTask


### MFCC FEATURES LIST 

In [5]:
# MFCC FEATURES LIST 

os.chdir(rootdir)
print("Path: ", os.getcwd())

classes = os.listdir(os.getcwd())

MFCC = []
FileNames = []
Label = []
n = -1

for x in classes:
    n += 1
    os.chdir(rootdir + "\\" + x)
    for audio_path in os.listdir(os.getcwd()):
        X , SR = librosa.load(audio_path)
        MFCC_Features = librosa.feature.mfcc(y = X, sr = SR)
        MFCC.append(MFCC_Features)
        Label.append(n)
        FileNames.append(audio_path)
        

Path:  C:\Users\Usman Naveed\Desktop\PAI Project\PAI_VolunteerTask


C:\Users\Usman Naveed\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\Users\Usman Naveed\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


### MAKING A MFCC LIST TO CSV

In [6]:
# MAKING A MFCC LIST TO DATAFRAME
os.chdir(path)
print("Location of MFCC.csv: ", os.getcwd())
df_mfcc = pd.DataFrame(MFCC)
df_mfcc.to_csv('MFCC.csv', index=False)

Location of MFCC.csv:  C:\Users\Usman Naveed\Desktop\PAI Project


C:\Users\Usman Naveed\anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


### WE CAN READ DATA FROM CSV TO HAVE MFCC FEATURES LIST

In [7]:
# READING A DATA FROM CSV TO HAVE MFCC FEATURES LIST
file = open("MFCC.csv", "r")
csv_reader = csv.reader(file)

MFCC_list = []
for row in csv_reader:
    MFCC_list.append(row)

print(MFCC_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### TAKING TRANSPOSE AND MEAN OF THE MFCC FEATURES LIST

In [25]:
# TAKING TRANSPOSE AND MEAN OF THE MFCC FEATURES LIST

MFCC_MEAN = []
for i in range(0,len(MFCC)):
    MFCC_MEAN.append(np.mean(MFCC[i].T, axis=0))
# MFCC_MEAN

### FLATTEN AND PADDING

In [9]:
# FLATTEN AND PADDING

FlattenValues = []
FlattenMean = []
maxVal=0
for i in range(0,len(MFCC)):
    FlattenValues.append(np.array(MFCC[i]).flatten())
    FlattenMean.append(np.array(MFCC_MEAN[i]).flatten())
    if FlattenValues[i].shape[0]>maxVal:
        maxVal=FlattenValues[i].shape[0]

PaddedValues = []
PaddedMean = []
for i in range(0,len(FlattenValues)):
    np.pad(FlattenValues[i], (0, maxVal-len(FlattenValues[i])))
    PaddedValues.append(np.pad(FlattenValues[i], (0, maxVal-len(FlattenValues[i]))))
    PaddedMean.append(np.pad(FlattenMean[i], (0, maxVal-len(FlattenMean[i]))))
    
# print(PaddedValues)
    
PaddedDF = pd.DataFrame({"Labels":Label, "MFCC Features":PaddedValues, "MFCC Mean":PaddedMean})
print(PaddedDF)

,Labels,MFCC Features,MFCC Mean
0,0,"[-717.5712, -717.5712, -717.0941, -717.1781, -...","[-491.33414, 116.10739, 5.0026007, 19.08632, -..."
1,0,"[-464.71918, -435.68018, -427.7529, -427.6081,...","[-357.92642, 71.06488, 22.784355, 15.4980135, ..."
2,0,"[-669.71497, -669.71497, -653.0248, -630.79724...","[-393.05588, 122.87493, 52.46836, -3.8200877, ..."
3,0,"[-559.966, -559.966, -504.3064, -353.51645, -3...","[-317.92526, 92.107704, 17.818577, 25.37494, 9..."
4,0,"[-479.9994, -451.33322, -440.08194, -439.05313...","[-379.72556, 60.72446, 38.965378, -8.665557, 0..."
...,...,...,...
6465,3,"[-529.7096, -527.4151, -529.144, -528.17236, -...","[-394.28625, 93.22102, 14.825175, 21.706749, 6..."
6466,3,"[-464.78415, -460.64105, -453.20917, -464.6736...","[-368.06393, 108.18454, 24.29488, 28.617796, 9..."
6467,3,"[-464.78415, -460.64105, -453.20917, -464.6736...","[-368.06393, 108.18454, 24.29488, 28.617796, 9..."
6468,3,"[-527.009, -527.4413, -526.5463, -523.66284, -...","[-368.76834, 112.900406, 17.835201, 14.849443,..."


### SKLEARN IMPORT LIBRARIES 

In [10]:
# SKLEARN IMPORT LIBRARIES 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### TRAIN_TEST_SPLIT()

In [11]:
# TRAIN_TEST_SPLIT()
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(PaddedDF["MFCC Mean"], PaddedDF["Labels"], test_size = 0.1)
print('Shape of X_train = ', X_train.shape)
print('Shape of y_train = ', y_train.shape)
print('Shape of X_test = ', X_test.shape)
print('Shape of y_test = ', y_test.shape)

Shape of X_train =  (5823,)
Shape of y_train =  (5823,)
Shape of X_test =  (647,)
Shape of y_test =  (647,)


### CONVERTING X_train and X_test into LIST

In [12]:
# CONVERTING X_train and X_test into LIST
X_train = list(X_train)
X_test = list(X_test)

### DECISION TREE

In [13]:
# DECISION TREE
from sklearn.tree import DecisionTreeClassifier  

dt_model = DecisionTreeClassifier(criterion='entropy', random_state=0)  
dt_model = dt_model.fit(X_train, y_train)

# PREDICTION 
pred = dt_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  63.214837712519326
F1 SCORE :  63.214837712519326
RECALL SCORE 63.214837712519326
PRECISION SCORE 63.214837712519326
CONFUSION MATRIX [[ 18   8   0  17]
 [ 20 270  16  51]
 [  5  20  16  13]
 [ 11  61  16 105]]


### K-NEAREST NEIGHBORS (KNN)

In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)

# PREDICTION
pred = knn_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  77.27975270479135
F1 SCORE :  77.27975270479135
RECALL SCORE 77.27975270479135
PRECISION SCORE 77.27975270479135
CONFUSION MATRIX [[ 27   7   2   7]
 [  6 320   3  28]
 [  2  15  24  13]
 [ 13  43   8 129]]


### LOGISTIC REGRESSION

In [21]:
# LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=6000)
lr_model.fit(X_train, y_train)

# PREDICTION
pred = lr_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  59.04173106646059
F1 SCORE :  59.04173106646059
RECALL SCORE 59.04173106646059
PRECISION SCORE 59.04173106646059
CONFUSION MATRIX [[  1  28   0  14]
 [  1 326   0  30]
 [  0  46   0   8]
 [  0 138   0  55]]


C:\Users\Usman Naveed\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### RANDOM FOREST CLASSIFIER

In [16]:
# RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(max_depth=50, random_state=0)
rfc_model.fit(X_train, y_train)

# PREDICTION
pred = rfc_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  73.41576506955178
F1 SCORE :  73.41576506955178
RECALL SCORE 73.41576506955178
PRECISION SCORE 73.41576506955178
CONFUSION MATRIX [[  7  23   0  13]
 [  0 351   0   6]
 [  0  36   2  16]
 [  0  78   0 115]]


### GRADIENT BOOSTING CLASSIFIER

In [17]:
# GRADIENT BOOSTING CLASSIFIER
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier()
gb_model.fit(X_train, y_train)

# PREDICTION
pred = gb_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  66.61514683153014
F1 SCORE :  66.61514683153014
RECALL SCORE 66.61514683153014
PRECISION SCORE 66.61514683153014
CONFUSION MATRIX [[  8  21   0  14]
 [  2 333   2  20]
 [  0  42   2  10]
 [  2 101   2  88]]


### SUPPORT VECTOR MACHINES

In [20]:
# SUPPORT VECTOR MACHINES
from sklearn import svm

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

# PREDICTION
pred = svm_model.predict(X_test)
print ("ACCURACY : ", accuracy_score(y_test,pred)*100)  
print("F1 SCORE : ", f1_score(y_test, pred, average = 'micro')*100)
print("RECALL SCORE", recall_score(y_test, pred, average = 'micro')*100)
print("PRECISION SCORE", precision_score(y_test, pred, average = 'micro')*100)
print("CONFUSION MATRIX", confusion_matrix(y_test, pred))

ACCURACY :  55.17774343122102
F1 SCORE :  55.17774343122102
RECALL SCORE 55.17774343122102
PRECISION SCORE 55.17774343122102
CONFUSION MATRIX [[  0  43   0   0]
 [  0 357   0   0]
 [  0  54   0   0]
 [  0 193   0   0]]


### GENERATING PICKLE FILES FOR ALL MODELS

In [22]:
# GENERATING PICKLE FILES FOR ALL MODELS
os.chdir(path)
print("Path: ", os.getcwd())


dt_model_file = "Decision Tree Model.pkl"
pickle.dump(dt_model, open(dt_model_file,'wb'))

knn_model_file = "K-Nearest Neighbor.pkl"
pickle.dump(knn_model, open(knn_model_file,'wb'))

rfc_model_file = "Random Forest Classifier.pkl"
pickle.dump(rfc_model, open(rfc_model_file,'wb'))

svm_model_file = "Support Vector Machine.pkl"
pickle.dump(svm_model, open(svm_model_file,'wb'))

gb_model_file = "Gradient Boosting.pkl"
pickle.dump(gb_model, open(gb_model_file,'wb'))

lr_model_file = "Linear Regression.pkl"
pickle.dump(lr_model, open(lr_model_file,'wb'))

print("pkl Files Created")

Path:  C:\Users\Usman Naveed\Desktop\PAI Project
pkl Files Created


In [27]:
# PREDICTION FOR A SINGLE DOWNLOADED FILE

# downloadFolderPath = "C:\\Users\\Usman Naveed\\Downloads"
# os.chdir(downloadFolderPath)
# print("Path: ", os.getcwd())
# fileName = "recorded.wav"

# mfccList = []
# X , SR = librosa.load(fileName)
# mfcc1 = librosa.feature.mfcc(y = X, sr = SR)
# mfcc1 = np.mean(mfcc1.T, axis=0)
# # mfcc1 = np.array(mfcc1.flatten())
# # mfcc1 = np.pad(mfcc1, (0, maxVal - len(mfcc1)))
# mfccList.append(mfcc1) 
# # print(mfcc1)

# def prediction_folder(p):
#     if p == 0:
#         print("A.C\n")
#     if p == 1:
#         print("Bulb-Light\n")
#     if p == 2:
#         print("Gaana-Song\n")
#     if p == 3:
#         print("T.V\n")

# print("\nPREDICTIONS:\n")
# pred = dt_model.predict(mfccList)
# print("Decision Tree: ", pred)
# prediction_folder(pred)

# pred = knn_model.predict(mfccList)
# print("K-Nearest Neighbor: ", pred)
# prediction_folder(pred)

# pred = rfc_model.predict(mfccList)
# print("Random Forest Classfier: ", pred)
# prediction_folder(pred)

# pred = svm_model.predict(mfccList)
# print("Support Vector Machines: ", pred)
# prediction_folder(pred)

# pred = gb_model.predict(mfccList)
# print("Gradient Boosting: ", pred)
# prediction_folder(pred)

# pred = lr_model.predict(mfccList)
# print("Linear Regression: ", pred)
# prediction_folder(pred)

# # FOR DELETING THE DOWNLOADED FILE
# os.remove(fileName)

Path:  C:\Users\Usman Naveed\Downloads

PREDICTIONS:

Decision Tree:  [1]
Bulb-Light

K-Nearest Neighbor:  [3]
T.V

Random Forest Classfier:  [1]
Bulb-Light

Support Vector Machines:  [1]
Bulb-Light

Gradient Boosting:  [1]
Bulb-Light

Linear Regression:  [3]
T.V

